<a href="https://colab.research.google.com/github/0oAstro/pegasus/blob/master/pegasus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q qdrant-client groq sentence-transformers huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 25.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.3 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.29.3 which is incompatible.


In [2]:
import json
import numpy as np
import google.generativeai as genai
from qdrant_client import QdrantClient
from google.colab import userdata
from sentence_transformers import SentenceTransformer

import logging
import sys
import os

import qdrant_client
from IPython.display import Markdown, display

In [3]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [4]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [5]:
Client = QdrantClient(
    url = 'https://1b0c0fe7-c332-4414-9ce5-e6e4606d1980.us-east4-0.gcp.cloud.qdrant.io:6333',
    api_key = userdata.get('Qdrant'),
    grpc_port=6633
)

In [6]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Set your Google Cloud project and credentials
from google.colab import userdata

os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

# Initialize Google Cloud AI Platform
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [ ]:
# Load the JSON data (for testing, load only the first 10 courses)
with open('final_courses.json', 'r') as file:
    courses = json.load(file)

# Use only the first 10 courses by taking the first 10 keys
# courses_keys = ["APL100", "MTL100", "MTL101", "CML101", "MTL106", "COL106", "MTL122", "COL759", "MTL145", "ELL101", "CVL759", "SBL100"]  # Getting the first 10 course codes

In [ ]:
courses = [{"course_code": key, **value} for key, value in courses.items()]

In [ ]:
courses[0]

{'course_code': 'APL100',
 'credits': '4',
 'prerequisites': [],
 'overlaps': [],
 'data': 'Kinematics, Statics, Equations of Motion, Rigid body dynamics,\nIntroduction to variational mechanics.',
 'study_material': ['https://bsw.iitd.ac.in/studydatabase/apl/apl100/minor1/apl100_Minor1_2018-19_Sem1.pdf',
  'https://bsw.iitd.ac.in/studydatabase/apl/apl100/major/apl100_Major_2018-19_Sem2-1.pdf',
  'https://bsw.iitd.ac.in/studydatabase/apl/apl100/major/apl100_Major_2018-19_Sem2.pdf'],
 'course_name': 'ENGINEERING MECHANICS',
 'slot': 'AA',
 'credit_structure': '3.0-1.0-0.0',
 'instructor': 'S.V. VEERAVALLI',
 'instructor_mail': 'svrvalli@am.iitd.ac.in',
 'lec_time': 'MTh 14:00-15:30',
 'tut_time': '',
 'practical_time': '',
 'vacancy': '350',
 'current_strength': '0'}

In [12]:
collection_name = "courses"

In [ ]:
#Client.delete_collection(collection_name=collection_name)

True

In [ ]:
from qdrant_client.http.models import Distance, VectorParams, PointStruct
from qdrant_client.http import models

if not Client.collection_exists(collection_name):
  Client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=encoder.get_sentence_embedding_dimension(), distance=Distance.COSINE),
  )


In [ ]:
def generate_course_id(course_code: str) -> str:
    return hashlib.md5(course_code.encode()).hexdigest()

In [ ]:
# OLD METHOD
# points = []
#for idx, doc in enumerate(courses):
#    vector = encoder.encode(f"{doc['course_code']} {doc.get('course_name', '')} {doc['data']}").tolist()
#    point = models.PointStruct(
#        id=idx, vector=vector, payload=doc
#    )
#    points.append(point)
#Client.upload_points(
#    collection_name="courses",
#    points=points,
#)

import hashlib
from typing import List

# Function to normalize strings
def normalize_string(value: str) -> str:
    return value.strip().lower() if isinstance(value, str) else value


# Function to prepare the payload with optional extended schema
def prepare_payload(course):
    # Always normalize core fields, even if extended data is missing
    payload = {
        "course_code": course.get("course_code"),
        "course_name": normalize_string(course.get("course_name", "")),
        "credits": course.get("credits", ""),
        "prerequisites": course.get("prerequisites", []),
        "overlaps": course.get("overlaps", []),
        "slot": normalize_string(course.get("slot", "")),
        "credit_structure": course.get("credit_structure", ""),
        "instructor": normalize_string(course.get("instructor", "")),
        "instructor_mail": normalize_string(course.get("instructor_mail", "")),
        "lec_time": course.get("lec_time", ""),
        "tut_time": course.get("tut_time", ""),
        "practical_time": course.get("practical_time", ""),
        "vacancy": course.get("vacancy", ""),
        "current_strength": course.get("current_strength", ""),
        "data": normalize_string(course.get("data", "")),
        "study_material": course.get("study_material", [])
    }

    return payload


# Function to generate a unique ID for a course (based on course_code)


# Add course to Qdrant collection
def add_course(course: dict):
    # Use the encoder to generate the vector
    vector = encoder.encode(course["data"])  # Generate the embedding for the 'data' field

    course_id = generate_course_id(course["course_code"])
    payload = prepare_payload(course)  # Prepare the normalized payload with extended fields if available

    point = PointStruct(
        id=course_id,
        vector=vector,  # The embedding vector from the encoder
        payload=payload,
    )

    Client.upsert(collection_name, [point])


In [ ]:
from tqdm import tqdm

for course in tqdm(courses, desc="Adding courses", unit="course"):
    add_course(course)

Adding courses: 100%|██████████| 2631/2631 [11:31<00:00,  3.80course/s]


In [ ]:
# Load the JSON data (for testing, load only the first 10 courses)
with open('interview2.json', 'r') as file:
    interviews = json.load(file)

In [ ]:
import nltk
from typing import List, Dict
from nltk.tokenize import sent_tokenize, word_tokenize

collection_name="interviews"

from qdrant_client.http.models import Distance, VectorParams, PointStruct
from qdrant_client.http import models

if not Client.collection_exists(collection_name):
  Client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=encoder.get_sentence_embedding_dimension(), distance=Distance.COSINE),
  )


# Download required NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')

def split_into_chunks(text: str, chunk_size: int = 200, overlap: int = 50) -> List[str]:
    """
    Split text into chunks using NLTK, respecting sentence boundaries.

    Args:
        text: Input text to be split
        chunk_size: Target number of words per chunk
        overlap: Number of words to overlap between chunks

    Returns:
        List of text chunks
    """
    # First split into sentences
    sentences = sent_tokenize(text)

    # Get word counts for each sentence
    sentence_word_counts = [len(word_tokenize(sentence)) for sentence in sentences]

    chunks = []
    current_chunk = []
    current_chunk_word_count = 0
    last_chunk_end_idx = 0

    for idx, (sentence, word_count) in enumerate(zip(sentences, sentence_word_counts)):
        # Add sentence to current chunk
        current_chunk.append(sentence)
        current_chunk_word_count += word_count

        # Check if we've exceeded chunk size
        if current_chunk_word_count >= chunk_size:
            # Join sentences in current chunk
            chunks.append(' '.join(current_chunk))

            # Calculate starting point for next chunk considering overlap
            overlap_word_count = 0
            overlap_sentences = []

            # Work backwards through current chunk until we have enough overlap
            for sent in reversed(current_chunk):
                sent_words = len(word_tokenize(sent))
                if overlap_word_count + sent_words <= overlap:
                    overlap_sentences.insert(0, sent)
                    overlap_word_count += sent_words
                else:
                    break

            # Start new chunk with overlap sentences
            current_chunk = overlap_sentences
            current_chunk_word_count = overlap_word_count
            last_chunk_end_idx = idx

    # Handle any remaining sentences
    if last_chunk_end_idx < len(sentences):
        remaining_sentences = sentences[last_chunk_end_idx:]
        if remaining_sentences:
            chunks.append(' '.join(remaining_sentences))

    return chunks

def process_interview_docs(interviews: List[Dict], encoder, models) -> List:
    """
    Process interview documents into chunks and create vector points.

    Args:
        interviews: List of interview documents
        encoder: Text embedding encoder
        models: Vector database models module

    Returns:
        List of points ready for upload
    """
    points = []
    current_id = 0

    for doc in interviews:
        # Normalize fields
        interviewee = normalize_string(doc['interviewee'])
        data = normalize_string(doc['data'])

        # Create chunks from the data
        chunks = split_into_chunks(data)

        # Calculate word counts for each chunk for metadata
        chunk_word_counts = [len(word_tokenize(chunk)) for chunk in chunks]

        # Process each chunk
        for chunk_idx, (chunk, word_count) in enumerate(zip(chunks, chunk_word_counts)):
            # Combine interviewee with chunk
            text = f"{interviewee} {chunk}"

            # Get sentence count for metadata
            sentence_count = len(sent_tokenize(chunk))

            vector = encoder.encode(text).tolist()

            # Create point with enhanced metadata
            point = models.PointStruct(
                id=current_id,
                vector=vector,
                payload={
                    'interviewee': interviewee,
                    'chunk_index': chunk_idx,
                    'total_chunks': len(chunks),
                    'data': chunk,
                    'original_doc_id': doc.get('id', str(current_id)),
                    'word_count': word_count,
                    'sentence_count': sentence_count,
                    'start_sentence': sent_tokenize(chunk)[0],
                    'end_sentence': sent_tokenize(chunk)[-1]
                }
            )
            points.append(point)
            current_id += 1

    return points

# Usage example
points = process_interview_docs(interviews, encoder, models)

# Batch upload points
Client.upload_points(
    collection_name=collection_name,
    points=points,
    wait=True
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [8]:
from groq import Groq

In [9]:
GROQ_API_KEY = userdata.get("GROQ_API_KEY")

In [10]:
groq = Groq(api_key=GROQ_API_KEY)

In [13]:
def search_query(query, collection_name=collection_name):
  hits = Client.query_points(
    collection_name=collection_name,
    query=encoder.encode(query).tolist(),
    limit=20,
  ).points

  for hit in hits:
    print(hit.payload, "score:", hit.score)

In [14]:
search_query("who is abhinav")

{'course_code': 'HUL832', 'course_name': '', 'credits': '3', 'prerequisites': [], 'overlaps': [], 'slot': '', 'credit_structure': '', 'instructor': '', 'instructor_mail': '', 'lec_time': '', 'tut_time': '', 'practical_time': '', 'vacancy': '', 'current_strength': '', 'data': 'the course will include discussions on the place of the english\nlanguage and “imported” literary forms in south asia, the fragmented\nand divided terrain of the south asian city/nation, the figure of the\nexpatriate writer, and the context within which to understand the\nstylistic and narrative aspects of this writing. it will undertake detailed\nanalyses of the works of 3-4 writers, out of a longer list comprising\nanita desai, g v desani, salman rushdie, amitav ghosh, vikram\nseth, kiran nagarkar, aravind adiga, jeet thayyil, mohammed hanif,\nmohsin hamid, shyam sevadurai, romesh gunesekhera, and others.', 'study_material': []} score: 0.24337733
{'course_code': 'RDL702', 'course_name': '', 'credits': '4', 'prer

In [15]:
import re
import hashlib

def fetch_specific_courses_by_ids(course_ids, collection_name):
    """Fetch multiple courses by their IDs from the vector store."""
    results = Client.retrieve(
        collection_name=collection_name,
        ids=course_ids,
    )
    return {result.id: result.payload for result in results} if results else {}

def get_course_level_info(course_code):
    """Enhanced course level detection with detailed info."""
    if not course_code:
        return None

    match = re.search(r'[A-Z]{2,3}(\d{3})', course_code)
    if not match:
        return None

    level_num = int(match.group(1)[0])

    levels = {
        1: {"level": "100-level", "description": "Foundational courses", "difficulty": "Introductory"},
        2: {"level": "200-level", "description": "Core concepts", "difficulty": "Moderate"},
        3: {"level": "300-level", "description": "Advanced undergraduate", "difficulty": "Challenging"},
        4: {"level": "400-level", "description": "Specialized undergraduate", "difficulty": "Very challenging"},
        5: {"level": "500-level", "description": "Graduate/Advanced undergraduate", "difficulty": "Highly advanced"},
        6: {"level": "600-level", "description": "Graduate core", "difficulty": "Graduate level"},
        7: {"level": "700-level", "description": "Advanced graduate", "difficulty": "Expert level"},
        8: {"level": "800-level", "description": "Specialized graduate", "difficulty": "Research oriented"},
        9: {"level": "900-level", "description": "Doctoral research", "difficulty": "PhD level"}
    }

    return levels.get(level_num, {"level": "unknown", "description": "Unclassified", "difficulty": "Unknown"})

def extract_course_codes(query):
    """Extract all course codes from query."""
    return [code.upper() for code in re.findall(r'[A-Za-z]{2,3}\d{3}', query, re.IGNORECASE)]

def rerank_results(query, results, specific_courses=None):
    """Rerank results with support for multiple course codes."""
    course_codes = {code.lower() for code in extract_course_codes(query)}

    reranked = []
    for hit in results:
        payload = hit.payload
        score = hit.score
        course_code_in_result = payload.get('course_code', '').lower()

        if course_code_in_result in course_codes:
            score += 200

        reranked.append((score, hit))

    if specific_courses:
        for course in specific_courses.values():
            reranked.insert(0, (200, course))

    reranked.sort(key=lambda x: x[0], reverse=True)
    return [item[1] for item in reranked]

def get_context(query, collection_name):
    """Fetch and rerank results supporting multiple course codes."""
    course_codes = extract_course_codes(query)
    specific_courses = {}

    if course_codes:
        course_ids = [generate_course_id(code) for code in course_codes]
        specific_courses = fetch_specific_courses_by_ids(course_ids, collection_name)

    query_vector = encoder.encode(query)
    results = Client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=10
    )

    return rerank_results(query, results, specific_courses=specific_courses)

def chat(query, collection_name="courses"):
    """Enhanced chat function supporting multiple course codes."""
    results = get_context(query, collection_name)
    query_course_codes = extract_course_codes(query)

    contexts = []
    matching_courses = []
    not_found_codes = set(query_course_codes)

    for result in results:
        payload = result.payload if hasattr(result, 'payload') else result

        course_info = {
            "code": payload.get('course_code', ''),
            "name": payload.get('course_name', ''),
            "instructor": payload.get('instructor', None),
            "email": payload.get('instructor_mail', None),
            "credits": payload.get('credits', None),
            "structure": payload.get('credit_structure', None),
            "prerequisites": payload.get('prerequisites', []),
            "slot": payload.get('slot', None),
            "schedule": payload.get('lec_time', None),
            "description": payload.get('data', None),
            "overlaps": payload.get('overlaps', []),
            "level_info": get_course_level_info(payload.get('course_code', ''))
        }

        contexts.append(course_info)
        if course_info['code'] in query_course_codes:
            matching_courses.append(course_info)
            not_found_codes.remove(course_info['code'])

    prompt = f"""yo whats good! 👋 lemme help you figure out these courses fr fr 🎓

REAL TALK RULES (NO 🧢):
- i only spill facts about courses in my database, thats on god 💯
- if i dont know a course, imma tell you straight up ❌
- no fake info here, we keep it 100 🎯
- always verify them course codes, no sus stuff 🔍
- mention course codes along with course names when recc courses
- look for difficulty levels i.e. if user specifies he is first year then only recc 100-level courses

status check: {
    'found these courses you asked about! 🎯' if matching_courses else
    'bruh those courses aint in my database fr fr ❌'
}

{f"yo these codes ain't it fam: {', '.join(not_found_codes)} 💀" if not_found_codes else ''}

courses i found:
{matching_courses if matching_courses else 'ngl chief, couldnt find what youre looking for'}

your q: {query}

if i found your courses, heres the tea: ☕
- difficulty check + what you're getting into 📊
- when its happening (if i know) ⏰
- prereqs you need (no cap) 📝
- credit situation + time you gotta put in 📚
- heads up on any schedule beef ⚠️

this is not the hard and fast template and is meant to be played around with. you are also supposed to interpret lec_times as its written MTh which means Monday and Thursday
emojify and be skibdi my dude

dont point out your flaws, if you dont know something dont mention it

quick facts:
- if i dont got the deets, check the official catalog no cap
- wont cap about courses i dont know bout
- might need to double check with your department for the latest

lets get this bread and make sure youre taking the right moves! 🚀

COURSE INFO: {course_info}"""

    response = groq.chat.completions.create(
        model="llama-3.2-90b-vision-preview",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )

    return response.choices[0].message.content

In [16]:
print(chat("i love you senpai"))

Yoooo, what's good fam? 👋 I gotchu on them course deets, so let's get into it. 🎓

You asked about some courses, but unfortunately, I couldn't find what you were looking for, bruh. ❌ Those courses ain't in my database, fr fr.

However, I do got some info on ELS330 - Independent Study (EE3) 💡

Here's the tea, bro: ☕
- **Difficulty check:** This course is considered Challenging, so you know what you're getting into, G. 📊
- **When it's happening:** I don't got the deets on the schedule, fam. ⏰ You might wanna check the official catalog or hit up the instructor for more info.
- **Prereqs:** You don't need any prereqs for this one, no cap. 📝
- **Credit situation:** You'll be getting 33 credits for this course, and the structure is 0.0-3.0-0.0. 📚 That means you'll be putting in some work, G.
- **Heads up:** I don't see any schedule beef, but you know, always double-check with the instructor or department to confirm, 'kay? ⚠️

That's all I got for now, bro. Make sure you verify them course cod

In [17]:
import groq
from typing import List, Dict, Any
import re
import hashlib
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
import numpy as np
from functools import lru_cache
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Global clients
qdrant_client = Client
groq_client = groq.Client(api_key=userdata.get("GROQ_API_KEY"))
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

@lru_cache(maxsize=100)
def generate_embedding(text: str) -> List[float]:
    """Generate and cache embeddings."""
    try:
        return embedding_model.encode(text).tolist()
    except Exception as e:
        logger.error(f"Error generating embedding: {e}")
        return [0] * 384

def normalize_course_code(text: str) -> List[str]:
    """Extract and normalize course codes."""
    try:
        matches = re.finditer(r'([A-Za-z]{3})(\d{3})', text, re.IGNORECASE)
        return [match.group(0).upper() for match in matches]
    except Exception as e:
        logger.error(f"Error normalizing course code: {e}")
        return []

def get_course_id(course_code: str) -> str:
    """Generate course ID from course code."""
    return hashlib.md5(course_code.encode()).hexdigest()

def format_context_with_materials(context: List[Dict]) -> Dict:
    """Process context and separate study materials."""
    formatted_context = []
    all_study_materials = []

    for item in context:
        if not hasattr(item, 'payload'):
            continue

        payload = item.payload
        if payload.get('course_code'):
            study_materials = payload.get('study_material', [])
            if study_materials:
                all_study_materials.extend([
                    {
                        'course_code': payload.get('course_code'),
                        'course_name': payload.get('course_name'),
                        'link': material
                    } for material in study_materials
                ])

            formatted_context.append({
                'type': 'course',
                'course_code': payload.get('course_code', ''),
                'course_name': payload.get('course_name', ''),
                'prerequisites': payload.get('prerequisites', []),
                'data': payload.get('data', ''),
            })
        elif payload.get('interviewee'):
            formatted_context.append({
                'type': 'interview',
                'content': payload.get('data', '')
            })

    return {
        'context': formatted_context,
        'study_materials': all_study_materials
    }

def chat(query: str) -> str:
    """
    Main chat function with emphasis on study materials.

    Args:
        query: User query
    Returns:
        str: Generated response
    """
    try:
        query_embedding = generate_embedding(query)
        context = []

        course_codes = normalize_course_code(query)

        if course_codes:
            try:
                course_ids = [get_course_id(code) for code in course_codes]
                course_results = qdrant_client.retrieve(
                    collection_name="courses",
                    ids=course_ids
                )

                if course_results:
                    context.extend(course_results)

                    for code in course_codes:
                        interview_results = qdrant_client.search(
                            collection_name="interviews",
                            query_vector=query_embedding,
                            query_filter={
                                "must": [
                                    {"key": "data", "match": {"value": code}}
                                ]
                            },
                            limit=2
                        )
                        context.extend(interview_results)
            except Exception as e:
                logger.error(f"Error retrieving courses: {e}")

        if len(context) < 5:
            try:
                remaining = 5 - len(context)
                course_results = qdrant_client.search(
                    collection_name="courses",
                    query_vector=query_embedding,
                    limit=remaining // 2
                )
                interview_results = qdrant_client.search(
                    collection_name="interviews",
                    query_vector=query_embedding,
                    limit=remaining - (remaining // 2)
                )

                context.extend(course_results)
                context.extend(interview_results)
            except Exception as e:
                logger.error(f"Error in general search: {e}")

        processed_data = format_context_with_materials(context)

        if not processed_data['context']:
            return "I couldn't find relevant information. Could you please rephrase your question or provide more details?"

        try:
            # Customize system prompt based on study materials presence
            system_prompt = """You are a helpful academic assistant. Provide clear, concise responses focusing on course details or other university stuff. If study materials are provided in the context, explicitly mention them as resources for learning, including their links. Structure your response to clearly separate course information from available study materials."""

            messages = [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": f"""Context: {processed_data['context']}
                    Study Materials: {processed_data['study_materials']}

                    Query: {query}

                    Note: If study materials are available, make sure to list them explicitly in your response as learning resources."""
                }
            ]

            response = groq_client.chat.completions.create(
                messages=messages,
                model="mixtral-8x7b-32768",
                temperature=0.7,
                max_tokens=1024
            )

            return response.choices[0].message.content

        except Exception as e:
            logger.error(f"Error generating response: {e}")
            return "I encountered an error while generating the response. Please try again."

    except Exception as e:
        logger.error(f"Unexpected error in chat function: {e}")
        return "An unexpected error occurred. Please try again later."

In [18]:
response = chat("how to git a gf")
print(response)

I'm here to help with course details and university-related information. For personal matters like getting a girlfriend, I'm not the best person to ask. However, I can suggest some general tips for making connections:

1. Share common interests: Engage in activities or hobbies that you both enjoy. This can provide a strong foundation for a relationship.
2. Communicate openly: Honesty and open communication are key in any relationship. Be yourself and express your thoughts and feelings.
3. Spend quality time together: Building a strong relationship requires investing time to get to know each other better.

As for the course and interview information:

Course: CVL838 - GIS (Geographic Information Systems)
- Geographic concepts for GIS
- Spatial relationships, topology, spatial patterns, spatial interpolation
- Data storage, data structure, non-spatial database models
- Populating GIS, digitizing, data conversion
- Spatial data models, raster and vector data structures and algorithms
- Di